# Sudoku Solver and Visualization

## Revise

In [ ]:
def revise(csp, var1, var2):
    removed = False
    # for each possible value remaining for the cell_i cell
    for value in csp["variables"][var1]:
        # if var1 is in conflict with var2 for each possibility in the domains
        if not any([value != poss for poss in csp["variables"][var2]]):
            # remove the value from var1's domain
            csp["variables"][var1] = remove(csp["variables"][var1], value)
            removed = True

    # returns true if the domain is smaller
    return removed

# helper function to remove a value from a domain
def remove(domain, value):
    new_domain = []
    # append each value in the domain if it isnt the one to be removed
    for d in domain:
        if d != value:
            new_domain.append(d)
    return new_domain



## AC3 Search

In [ ]:
def AC3(csp):
    # initialize queue with every arc in the constraints
    queue = [(xi, xj) for xi, xj in csp["constraints"].keys()]
    # while the queue isnt empty
    while queue:
        # pop an arc from the queue
        (xi, xj) = queue.pop(0)
        # if revise made the domain smaller
        if revise(csp, xi, xj):
            # exit if the domain is empty, no solution exists
            if len(csp["variables"][xi]) == 0:
                return False
            # for each xk in xi's neighbors - {xj}
            for xk in get_neighbors(csp, xi, xj):
                if xk != xj:
                    # add the arc to the queue
                    queue.append((xk, xi))
    return True


# helper function to get the neighbors of a given var
def get_neighbors(CSP, var, xj):
    neighbors = []
    # go through every constraint to find each one with var in it
    for (v1, v2) in CSP['constraints']:
        if v1 == var:
            neighbors.append(v2)
        elif v2 == var:
            neighbors.append(v1)
    return neighbors


## Search Heuristic

In [ ]:
def minimum_remaining_values(CSP, vars):
    # store the unassigned variables in a dict
    unassigned_variables = {}

    # go through each cell and check if it is assigned
    for square in CSP["variables"]:
        if square not in vars:
            # add it to the list with its length
            unassigned_variables[square] = len(CSP["variables"][square])

    # get the smallest length
    mrv = min(unassigned_variables, key=unassigned_variables.get)
    return mrv



## Backtracking Search

*   To run this, in colab, make sure you upload the constraint9x9.py file
*   To run on your system, make sure the constraint9x9.py file is in the same directory as the notebook, uncomment line 1, and comment out lines 4, 5, and 6



In [ ]:
#from sudoku_constraints9x9 import constraint9x9
import sys
import copy
constraint9x9_file = '/content/sudoku_constraint9x9.py'
sys.path.append(constraint9x9_file)
import sudoku_constraints9x9


def backtrack(csp):
    # if assignment is complete, print and return it
    if is_complete(csp):
        print('Solution Found!')
        printSolution(csp)
        return [csp["assignment"], csp["order_assigned"], csp["remaining_unassigned"]]

    # select unassigned variable using mrv
    var = minimum_remaining_values(csp, csp["assignment"])

    # make a deep copy of the domain to revert to if a solution isnt found
    original_domain = copy.deepcopy(csp["variables"])

    # for each value in the domain values
    for value in csp["variables"][var]:
        # check if the value is consistent with the rest of the domain using AC3
        csp["variables"][var] = [value]
        if AC3(csp):
            # if it is consistent, add it to the assignment
            csp["assignment"][var] = value
            csp["order_assigned"].append(var)

            # store the domain after insertion
            current_domain_snapshot = copy.deepcopy(csp["variables"])
            csp["remaining_unassigned"].append(current_domain_snapshot)

            # recursively call with new values
            result = backtrack(csp)

            # if a solution was found, return it
            if result:
                return result
            csp["order_assigned"].pop()
            csp["remaining_unassigned"].pop()
        # revert to the original domain if the assignment didnt return a solution
        csp["variables"] = copy.deepcopy(original_domain)

        # remove the value from assignment since no solution was found with it
        if var in csp["assignment"]:
            del csp["assignment"][var]

    # return false when no solution is found
    return False



def is_complete(CSP):
    # check that every variable is assigned, no need to check for consistency since it was done at every step
    return set(CSP["assignment"].keys()) == set(CSP["variables"].keys())



# helper function to turn 2d puzzle list into 3d list of domains
def prepare_puzzle(puzzle):
    # get the size of the NxN puzzle
    size = len(puzzle)

    # get the entire domain of the puzzle
    nums = []
    for i in range(1,  size + 1):
        nums.append(i)

    # initialize list to hold each variable domain
    domain = []

    # go through each variable, and if the square is empty, set it to the entire domain of the puzzle (nums)
    # otherwise, set assign the domain to the number at the index
    for i in range(len(puzzle)):
        domain.append([])
        for j in range(len(puzzle[i])):
            if puzzle[i][j] is None:
                domain[i].append(nums)
            else:
                domain[i].append([puzzle[i][j]])
    # return the new 3d list of domains
    return domain




# helper function to turn a constraint list and 3d list of domains into a CSP
def turn_puzzle_into_csp(constraints, puzzle):
    # make new dict
    csp = {}
    # add given constraints
    csp["constraints"] = constraints

    # add each variable from the given assignment
    csp["variables"] = {}
    csp["assignment"] = {}
    csp["order_assigned"] = []
    csp["remaining_unassigned"] = []

    i = 1
    j = 1
    for i in range(len(puzzle)):
        for j in range(len(puzzle[i])):
            csp["variables"]['C'+str(i + 1) + str(j + 1)] = puzzle[i][j]

    # if using the 4x4 constraints that I defined (9x9 has size of 810)
    if len(csp["constraints"]) != 810:
        csp["constraints"] = constraints["constraints"]
        csp["variables"] = constraints["variables"]
    return csp




# print the solution like an actual sudoku board
def printSolution(csp):
    # print the order each variable was assigned and the domains of the remaining unassigned variables
    print('Order each variable was assigned: ')
    for i in range(len(csp["order_assigned"])):

        # if it ends in 1 or 2 change the suffix accordingly
        suffix = 'th'
        if (i + 1) % 10 == 1:
            suffix = 'st'
        elif (i + 1) % 10 == 2:
            suffix = 'nd'

        print('Inserted ' + str(i + 1) + suffix + ': ' + str(csp["order_assigned"][i]))
        print('Remaining domains after ' + str(csp["order_assigned"][i]) + ' was assigned')
        print(csp["remaining_unassigned"][i])
        print()

    # size is the square root of the number of variables
    size = int(len(csp["variables"].keys()) ** 0.5)
    # define 2d list to place each assignment
    solution = [['X']*size for _ in range(size)]

    # count to keep track of how many values have been printed on each line
    count = 0
    # format the lines underneath the values
    num_spaces = 6 * int(len(csp["variables"]) ** 0.5)

    # for each key, get the index and add it to the solution list
    for key in csp["assignment"].keys():
        x = int(key[1]) - 1
        y = int(key[2]) - 1
        solution[x][y] = csp["assignment"][key]

    # print everything out from solution
    for i in range(int(len(csp["variables"].keys()) ** 0.5)):
        for j in range(int(len(csp["variables"].keys()) ** 0.5)):
            count += 1
            if solution[i][j] != ' ':
                print('| ' + str(solution[i][j]) + ' |', end= " ")
            else:
                print('| X |', end= " ")
            if count == int(len(csp["variables"]) ** 0.5):
                count = 0
                print()
                print('-' * num_spaces)




def main():
    # initialize each puzzle
    puzzle_1 = [
        [7, None, None, 4, None, None, None, 8, 6],
        [None, 5, 1, None, 8, None, 4, None, None],
        [None, 4, None, 3, None, 7, None, 9, None],
        [3, None, 9, None, None, 6, 1, None, None],
        [None, None, None, None, 2, None, None, None, None],
        [None, None, 4, 9, None, None, 7, None, 8],
        [None, 8, None, 1, None, 2, None, 6, None],
        [None, None, 6, None, 5, None, 9, 1, None],
        [2, 1, None, None, None, 3, None, None, 5]
    ]
    puzzle_2 = [
        [1, None, None, 2, None, 3, 8, None, None],
        [None, 8, 2, None, 6, None, 1, None, None],
        [7, None, None, None, None, 1, 6, 4, None],
        [3, None, None, None, 9, 5, None, 2, None],
        [None, 7, None, None, None, None, None, 1, None],
        [None, 9, None, 3, 1, None, None, None, 6],
        [None, 5, 3, 6, None, None, None, None, 1],
        [None, None, 7, None, 2, None, 3, 9, None],
        [None, None, 4, 1, None, 9, None, None, 5]
    ]
    puzzle_3 = [
            [1, None, None, 8, 4, None, None, 5, None],
            [5, None, None, 9, None, None, 8, None, 3],
            [7, None, None, None, 6, None, 1, None, None],
            [None, 1, None, 5, None, 2, None, 3, None],
            [None, 7, 5, None, None, None, 2, 6, None],
            [None, 3, None, 6, None, 9, None, 4, None],
            [None, None, 7, None, 5, None, None, None, 6],
            [4, None, 1, None, None, 6, None, None, 7],
            [None, 6, None, None, 9, 4, None, None, 2]
    ]
    puzzle_4 = [
        [None, None, None, None, 9, None, None, 7, 5],
        [None, None, 1, 2, None, None, None, None, None],
        [None, 7, None, None, None, None, 1, 8, None],
        [3, None, None, 6, None, None, 9, None, None],
        [1, None, None, None, 5, None, None, None, 4],
        [None, None, 6, None, None, 2, None, None, 3],
        [None, 3, 2, None, None, None, None, 4, None],
        [None, None, None, None, None, 6, 5, None, None],
        [7, 9, None, None, 1, None, None, None, None]
    ]
    puzzle_5 = [
        [None, None, None, None, None, 6, None, 8, None],
        [3, None, None, None, None, 2, 7, None, None],
        [7, None, 5, 1, None, None, 6, None, None],
        [None, None, 9, 4, None, None, None, None, None],
        [None, 8, None, None, 9, None, None, 2, None],
        [None, None, None, None, None, 8, 3, None, None],
        [None, None, 4, None, None, 7, 8, None, 5],
        [None, None, 2, 8, None, None, None, None, 6],
        [None, 5, None, 9, None, None, None, None, None]
    ]

    # make sure the puzzles are compatible with the functions
    puzzle_1 = prepare_puzzle(puzzle_1)
    puzzle_2 = prepare_puzzle(puzzle_2)
    puzzle_3 = prepare_puzzle(puzzle_3)
    puzzle_4 = prepare_puzzle(puzzle_4)
    puzzle_5 = prepare_puzzle(puzzle_5)

    # combine puzzle constraints and variables into CSP map
    CSP1 = turn_puzzle_into_csp(sudoku_constraints9x9.constraint9x9, puzzle_1)
    CSP2 = turn_puzzle_into_csp(sudoku_constraints9x9.constraint9x9, puzzle_2)
    CSP3 = turn_puzzle_into_csp(sudoku_constraints9x9.constraint9x9, puzzle_3)
    CSP4 = turn_puzzle_into_csp(sudoku_constraints9x9.constraint9x9, puzzle_4)
    CSP5 = turn_puzzle_into_csp(sudoku_constraints9x9.constraint9x9, puzzle_5)

    # iterate through each CSP to find a solution
    CSPS = [CSP1, CSP2, CSP3, CSP4, CSP5]
    for i in range(5):
        print('Looking for a solution for Puzzle', str(i + 1) + ': ')
        result = backtrack(CSPS[i])
        if not result:
            print('No Solution Found')


if __name__ == "__main__":
    main()

Looking for a solution for Puzzle 1: 
Solution Found!
Order each variable was assigned: 
Inserted 1st: C11
Remaining domains after C11 was assigned
{'C11': [7], 'C12': [2, 3, 7, 9], 'C13': [2, 3, 7], 'C14': [4], 'C15': [1, 4, 9], 'C16': [1, 4, 5, 9], 'C17': [2, 3, 5], 'C18': [8], 'C19': [6], 'C21': [6, 7, 9], 'C22': [5], 'C23': [1], 'C24': [2, 5, 6], 'C25': [8], 'C26': [1, 5, 8, 9], 'C27': [4], 'C28': [2, 3, 4, 5, 7, 8], 'C29': [1, 2, 3, 4, 6, 7], 'C31': [1, 5, 6, 8], 'C32': [4], 'C33': [1, 2, 5, 8], 'C34': [3], 'C35': [1, 3, 4, 6, 8], 'C36': [7], 'C37': [2, 3, 4, 5, 6, 8], 'C38': [9], 'C39': [1, 2, 3, 4, 6, 7, 9], 'C41': [3], 'C42': [2, 3, 5, 7], 'C43': [9], 'C44': [3, 4, 5, 7, 8], 'C45': [3, 4, 7, 8], 'C46': [6], 'C47': [1], 'C48': [2, 3, 4, 5, 9], 'C49': [1, 2, 3, 4, 6, 9], 'C51': [1, 3, 5, 6, 7, 8, 9], 'C52': [3, 5, 6, 7, 9], 'C53': [1, 3, 5, 7, 8, 9], 'C54': [3, 4, 5, 6, 7, 8], 'C55': [2], 'C56': [1, 4, 5, 6, 7, 8], 'C57': [1, 2, 3, 4, 5, 6], 'C58': [2, 3, 4, 5, 9], 'C59': [1, 2, 

## Web Server

In [ ]:
!pip install flask
from flask import Flask, render_template, request

!mkdir templates
!wget -O /content/templates/index.html https://raw.githubusercontent.com/ddeflores/sudoku/main/templates/index.html
!wget -O /content/templates/solved.html https://raw.githubusercontent.com/ddeflores/sudoku/main/templates/solved.html
!wget https://raw.githubusercontent.com/ddeflores/sudoku/main/sudoku_constraints9x9.py

from sudoku_constraints9x9 import constraint9x9

from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

app = Flask(__name__)

# initialize each puzzle
puzzle_1 = [
    [7, None, None, 4, None, None, None, 8, 6],
    [None, 5, 1, None, 8, None, 4, None, None],
    [None, 4, None, 3, None, 7, None, 9, None],
    [3, None, 9, None, None, 6, 1, None, None],
    [None, None, None, None, 2, None, None, None, None],
    [None, None, 4, 9, None, None, 7, None, 8],
    [None, 8, None, 1, None, 2, None, 6, None],
    [None, None, 6, None, 5, None, 9, 1, None],
    [2, 1, None, None, None, 3, None, None, 5]
]
puzzle_2 = [
    [1, None, None, 2, None, 3, 8, None, None],
    [None, 8, 2, None, 6, None, 1, None, None],
    [7, None, None, None, None, 1, 6, 4, None],
    [3, None, None, None, 9, 5, None, 2, None],
    [None, 7, None, None, None, None, None, 1, None],
    [None, 9, None, 3, 1, None, None, None, 6],
    [None, 5, 3, 6, None, None, None, None, 1],
    [None, None, 7, None, 2, None, 3, 9, None],
    [None, None, 4, 1, None, 9, None, None, 5]
]
puzzle_3 = [
    [1, None, None, 8, 4, None, None, 5, None],
    [5, None, None, 9, None, None, 8, None, 3],
    [7, None, None, None, 6, None, 1, None, None],
    [None, 1, None, 5, None, 2, None, 3, None],
    [None, 7, 5, None, None, None, 2, 6, None],
    [None, 3, None, 6, None, 9, None, 4, None],
    [None, None, 7, None, 5, None, None, None, 6],
    [4, None, 1, None, None, 6, None, None, 7],
    [None, 6, None, None, 9, 4, None, None, 2]
]
puzzle_4 = [
    [None, None, None, None, 9, None, None, 7, 5],
    [None, None, 1, 2, None, None, None, None, None],
    [None, 7, None, None, None, None, 1, 8, None],
    [3, None, None, 6, None, None, 9, None, None],
    [1, None, None, None, 5, None, None, None, 4],
    [None, None, 6, None, None, 2, None, None, 3],
    [None, 3, 2, None, None, None, None, 4, None],
    [None, None, None, None, None, 6, 5, None, None],
    [7, 9, None, None, 1, None, None, None, None]
]
puzzle_5 = [
    [None, None, None, None, None, 6, None, 8, None],
    [3, None, None, None, None, 2, 7, None, None],
    [7, None, 5, 1, None, None, 6, None, None],
    [None, None, 9, 4, None, None, None, None, None],
    [None, 8, None, None, 9, None, None, 2, None],
    [None, None, None, None, None, 8, 3, None, None],
    [None, None, 4, None, None, 7, 8, None, 5],
    [None, None, 2, 8, None, None, None, None, 6],
    [None, 5, None, 9, None, None, None, None, None]
]

empty_board = [[None]*9 for _ in range(9)]

@app.route("/")
def sudoku():
    return render_template('index.html', empty_board=empty_board, board0=puzzle_1, board1=puzzle_2, board2=puzzle_3, board3=puzzle_4, board4=puzzle_5)

@app.route("/solve", methods=['POST'])
def solve():
    # form request when button is clicked
    form = request.form.to_dict()
    # parse the data
    index = list(form.keys())[0]
    # empty 9x9 board
    board = [[None for _ in range(9)] for _ in range(9)]

    # Fill the board with the values from the form
    if index[5] == '0':
        board = puzzle_1
    elif index[5] == '1':
        board = puzzle_2
    elif index[5] == '2':
        board = puzzle_3
    elif index[5] == '3':
        board = puzzle_4
    elif index[5] == '4':
        board = puzzle_5
    else:
        # if the puzzle is given from user input, fill in the board based on the data
        for key in form.keys():
            x = int(key[4])
            y = int(key[6])
            if form.get(key) != '':
                board[x][y] = form.get(key, None)

    # combine puzzle constraints and variables into CSP
    to_solve = turn_puzzle_into_csp(constraint9x9, prepare_puzzle(board))
    solved = backtrack(to_solve)

    # if there is a solution, get the data ready to be displayed (convert back to lists)
    if solved:
        solution = [[' ']*9 for _ in range(9)]
        for key in solved[0].keys():
            x = int(key[1]) - 1
            y = int(key[2]) - 1
            solution[x][y] = solved[0][key]
        remaining = getRemaining(solved[2])
        order = solved[1]
    else:
        # if there is no solution, reset everything
        solution = [[' ']*9 for _ in range(9)]
        remaining = []
        order = []
    return render_template('solved.html', solved=solution, remaining=remaining, order=order)

# helper function to preprocess the remaining domains at each step
def getRemaining(remaining):
    boards = []
    for board in remaining:
        current_board = [[' ']*9 for _ in range(9)]
        for var in board.keys():
            x = int(var[1]) - 1
            y = int(var[2]) - 1
            if len(board[var]) == 1:
                current_board[x][y] = board[var][0]
        boards.append(current_board)
    return boards

app.run()


mkdir: cannot create directory ‘templates’: File exists
--2024-04-22 16:08:22--  https://raw.githubusercontent.com/ddeflores/sudoku/main/templates/index.html
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5300 (5.2K) [text/plain]
Saving to: ‘/content/templates/index.html’

/content/templates/ 100%[===================>]   5.18K  --.-KB/s    in 0s      

2024-04-22 16:08:22 (24.8 MB/s) - ‘/content/templates/index.html’ saved [5300/5300]

--2024-04-22 16:08:22--  https://raw.githubusercontent.com/ddeflores/sudoku/main/templates/solved.html
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


# Steps to run the web server

- pip install flask
- make sure the interpreter you are using is the same version you installed flask in! ( you can check this by running the command pip show flask)
- double check that the constraint9x9.py file is in the same directory
- make sure that the index.html and solved.html files are in a folder called "templates" that is located in the same directory
- run the program, and click the link in the terminal
    - if on colab, click the link that looks like this:
        - https://9xc5b1540hs-496ff2e9c6d22116-5000-colab.googleusercontent.com/
    - otherwise, choose the localhost link:
        - http://127.0.0.1:5000